In [3]:
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import copy
from gtfparse import read_gtf
import os
import pyranges as pr


In [4]:
gtf_ = pr.read_gtf("/nfs/genomes/human_hg38_dec13_no_random/gtf/Homo_sapiens.GRCh38.99.chr.canonical.gtf") #dowload gtf file from ensembl
gtf_ = gtf_.df

In [16]:
genes = [ "EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

gtf_xy = gtf_[gtf_.gene_name.isin(genes)]

#extract the exons and genes
gtf_xy = gtf_xy[(gtf_xy['Feature'] == "CDS")]

gtf_xy['Feature'].unique() 
gtf_xy['gene_name'].unique()
gtf_xy['Feature'].unique() 

array(['CDS'], dtype=object)

In [17]:
def intron_overlapping(exons, intron_start, intron_end):
      
    for index in range(len(exons)): 
        exon_st = exons.iloc[index]['Start']
        exon_end = exons.iloc[index]['End']
        
        if ((exon_st >= intron_start) & (exon_st <= intron_end)):
            return True
        if ((exon_end >= intron_start) & (exon_end <= intron_end)):
            return True
        
        if ((exon_st <= intron_start) & (exon_end >= intron_start)):
            return True 
        if ((exon_st <= intron_end) & (exon_end >= intron_end)):
            return True

    return False


In [18]:
intron_coordinates = pd.DataFrame(columns=['seqname', 'feature', 'start', 'end', 'gene_name'])  
genes = [ "EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

#genes = ["ZFY"] 

for gene in genes: 
    testzfx = gtf_xy[gtf_xy['gene_name'] == gene] 
    exons = testzfx[testzfx['Feature'] == "CDS"]
    
    exons = exons.sort_values(['Chromosome', 'Start', 'End']) #sort values start to end 
    grouped_exons = exons.groupby('transcript_id')
    print(grouped_exons)
    for name, group in grouped_exons:
      #  print(name)
        for i in range(1, len(group)):
            intron_start = group.iloc[i-1][4] 
            intron_end = group.iloc[i][3] 
            
            if (intron_end - intron_start > 200) & (intron_end > intron_start): 
                intron_start = intron_start + 100
                intron_end = intron_end - 100
            else:
                continue
        # Check for overlap with exons of any transcripts 
            #get rid of any grouped exon 
            if (group.iloc[i-1][24] == '1') | (group.iloc[i][24] == '1'): 
                print('FIRST', intron_start, intron_end)
                continue
            
            if intron_overlapping(exons, intron_start, intron_end):                 
                print("HERE", intron_start, intron_end) 
                continue
            
            else:
                print("adding", intron_start, intron_end) 
                new_row = pd.Series({'seqname': testzfx['Chromosome'].iloc[0], 'feature': 'intron', 'start': intron_start, 'end':intron_end, 'gene_name':gene}) 
                intron_coordinates = intron_coordinates.append(new_row, ignore_index=True) 
                

adding 20128411 20130415
adding 20130707 20132081
adding 20132363 20133856
adding 20134107 20135637
HERE 20135941 20141524
adding 20128411 20130415
adding 20130707 20132081
adding 20132363 20133856
adding 20134107 20135637
adding 20135941 20138438
adding 20138722 20141524
adding 20575987 20579507
adding 20579791 20582489
adding 20582793 20584373
adding 20584624 20587923
adding 20588205 20589383
adding 20589675 20592240
adding 20575987 20579507
adding 20579791 20582489
HERE 20582793 20587923
adding 20588205 20589383
adding 20589675 20592240


/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

adding 19706963 19707046
adding 19708523 19708774
adding 19709113 19709350
adding 19709909 19710275
adding 19710572 19715251


/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

adding 19716104 19716178
adding 19716815 19720771
HERE 19721411 19731671
adding 19732265 19732483
adding 19732842 19735251
adding 19735850 19739427


...

/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

adding 41196459 41196491
adding 41196838 41197263
adding 41197610 41198427
adding 41198850 41200959
adding 41201380 41205202
adding 41205593 41210408
adding 41210782 41214467
adding 41214809 41215798


/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

adding 41216752 41217119
adding 41217443 41218271
adding 41218697 41219001
adding 41219331 41223116
adding 41223502 41224641
adding 41225237 41229152
adding 41229879 41230400
adding 41230696 41232286
HERE 2841727 2842064
adding 2842342 2843976
adding 2844357 2845545
adding 2845843 2854499
adding 2854871 2864987
adding 2865345 2866692
adding 2842342 2843976
adding 2844357 2845545
adding 2845843 2854499
adding 2854871 2864987
adding 2865345 2866692
adding 72272872 72273131
adding 72273489 72273700
adding 72274072 72274952
adding 72275250 72275443
adding 72275824 72276056
adding 72276334 72277092


/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_2357504/3447769798.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

In [20]:
gtf_small = gtf_xy[['Chromosome', 'Feature', 'Start', 'End', 'gene_name']] 
gtf_small = gtf_small.rename(columns={"Chromosome": "seqname", "Feature": "feature", "Start":"start", "End": "end", "gene_name": "gene_name"}) #("seqname", "feature", "start", "end", "gene_name")
gtf_small[gtf_small['gene_name'] == 'DDX3X']

,seqname,feature,start,end,gene_name
2813677,chrX,CDS,41334252,41334297,DDX3X
2813680,chrX,CDS,41337407,41337465,DDX3X
2813682,chrX,CDS,41339035,41339083,DDX3X
2813684,chrX,CDS,41341483,41341616,DDX3X
2813686,chrX,CDS,41342494,41342653,DDX3X
...,...,...,...,...,...
2815274,chrX,CDS,41346319,41346410,DDX3X
2815276,chrX,CDS,41346504,41346622,DDX3X
2815278,chrX,CDS,41346858,41347012,DDX3X
2815280,chrX,CDS,41347311,41347451,DDX3X


In [21]:
vertically_concatenated = pd.concat([gtf_small, intron_coordinates], axis=0) 
vertically_concatenated = vertically_concatenated[(vertically_concatenated['feature'] == "CDS") | (vertically_concatenated['feature'] == "intron") | ((vertically_concatenated['feature'] == 'exon') )]

In [4]:
vertically_concatenated['feature'].unique()
vertically_concatenated.to_csv(myPath + "/tables/" + "XY_gtf_w_introns_012924.txt",  index=None, sep=str('\t'))

NameError: name 'vertically_concatenated' is not defined